### 접근
- 검은 점과 흰 점으로 볼록껍질을 각각 구한 뒤, 그 직선끼리 닿는지 여부를 파악한다.
- 닿지 않는다면 분리할 수 있는 것일 것이다.
- 꼭짓점의 개수 n+m개에 대해서 $O((n+m)^2)$만큼 걸릴 것 같다.
  - 그런데 어떤 볼록껍질이 다른 볼록껍질을 완전히 포함하는 경우가 있는데, 이 경우를 처리할 수 없다..

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

ccw = lambda o, a, b: (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
def cross(a, b) :
  t1 = ccw(a[0], a[1], b[0]) * ccw(a[0], a[1], b[1])
  t2 = ccw(b[0], b[1], a[0]) * ccw(b[0], b[1], a[1])
  if t1 < 0 and t2 < 0 : return True
  if t1 == 0 and t2 == 0: return b[0] <= a[1] and a[0] <= b[1]
  return t1 <= 0 and t2 <= 0

def andrews(P):
  P = sorted(P)

  lo = []
  for p in P:
    while len(lo) >= 2 and ccw(lo[-2], lo[-1], p) <= 0:
      lo.pop()
    lo.append(p)
  lo.pop()

  up = []
  for p in reversed(P) :
    while len(up) >= 2 and ccw(up[-2], up[-1], p) <= 0:
      up.pop()
    up.append(p)
  up.pop()
  
  return lo + up

def sol() :
  N, M = map(int, input().split())
  H1 = andrews([tuple(map(int, input().split())) for _ in range(N)])
  H2 = andrews([tuple(map(int, input().split())) for _ in range(M)])
  

for _ in range(int(input())) :
  sol()

- 볼록다각형 내부 점 판정문제부터 풀고 다시 재개